### Deep Learning in the Eye Tracking World 
#### the tutorial presented during ETRA 2021 (https://etra.acm.org/2021/acceptedtutorials.html)
#### the code downloaded from: https://github.com/kasprowski/etra2021
@author: pawel@kasprowski.pl

In [1]:
import os
import cv2
import numpy as np

from sklearn.model_selection._split import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM

if not os.path.exists("animals_img"):
    os.makedirs("animals_img")

## Download the dataset

In [ ]:
import requests
import zipfile
#r = requests.get("https://raw.githubusercontent.com/kasprowski/etra2021/main/emvic/emvic.data", allow_redirects=True)
r = requests.get("http://www.kasprowski.pl/tutorial/animals.zip", allow_redirects=True)
open('animals.zip', 'wb').write(r.content)
print("Downloaded animals.zip")
with zipfile.ZipFile("animals.zip","r") as zip_ref:
    zip_ref.extractall("animals")
print("Uzipped to /animals directory")

## Load data and convert it to sequences

In [8]:
def load_files(indir):
    samples = []
    for file in os.listdir(indir):
        if file.endswith("csv"):
            samples.append(np.genfromtxt(os.path.join(indir, file), delimiter=','))
    samples = np.array(samples)
    return samples

# Turns one sample of length N into (N-sequence_dim) samples of length sequence_dim
# (only if sequence_lag==1!)
def make_sequences(samples, sequence_dim = 20, sequence_lag = 1):
    nsamples = []
    nlabels = []
    for sample in samples:
        for i in range(0,len(sample)-sequence_dim-1,sequence_lag):
            nsample = np.zeros((sequence_dim,2))
            for j in range(i,i+sequence_dim):
                nsample[j-i,0] = sample[j,1]
                nsample[j-i,1] = sample[j,2]
            nsamples.append(nsample)
            nlabels.append(sample[i+sequence_dim,1:])
    samples = np.array(nsamples, dtype="float")
    labels = np.array(nlabels)
    return samples,labels

samples = load_files("animals")
sequence_dim=20
sequence_lag=1
    
samples, labels = make_sequences(samples, sequence_dim, sequence_lag)
print("Samples:",samples.shape)
print("Labels:",labels.shape)


Samples: (5757, 20, 2)
Labels: (5757, 2)


<ipython-input-8-d80f17cc5c58>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  samples = np.array(samples)


## Create a model

In [4]:
model = Sequential()
model.add(LSTM(128,input_shape=(sequence_dim,2),return_sequences=True))
model.add(LSTM(128))
model.add(Dense(64))
model.add(Dense(2))
model.compile(loss="mean_absolute_error", optimizer="adam", metrics=["mae"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 128)           67072     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 207,042
Trainable params: 207,042
Non-trainable params: 0
_________________________________________________________________
None


## Divide the dataset into training and test

In [5]:
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels, test_size=0.15, random_state=42)

## Create the ground truth image

In [9]:
imname = "animal-11"    
image = cv2.imread("animals/{}.jpg".format(imname))
# create the ground truth image with all train gazes
for j in range(len(trainLabels)):
    s = trainLabels[j]
    cv2.circle(image,(int(s[0]),int(s[1])),10,(255,0,0),3)
cv2.imwrite("animals_img/{}_truth.jpg".format(imname),image)
    
print("Image saved to /animals_img")    

Image saved to /animals_img


## Build the model and save the results after each epoch

In [11]:
EPOCHS = 30
for e in range(EPOCHS):
    print("="*50)
    print("Iteration: {}".format(e))
    model.fit(trainSamples, trainLabels, validation_data=(testSamples, testLabels), epochs=1
              , batch_size=128, verbose=1)

    predictions = model.predict(testSamples)

    # create and save image with all current predictions
    image = cv2.imread("animals/{}.jpg".format(imname))
    cv2.line(image,(0,0),(200,200),(255,255,255),2)
    for p in predictions:    
        cv2.circle(image,(int(p[0]),int(p[1])),10,(0,255,0),3)
    cv2.imwrite("animals_img/{}_e{:02d}.jpg".format(imname,e),image)

model.save("model_rnn.h5")

Iteration: 0
39/39 [==============================] - 2s 59ms/step - loss: 702.0654 - mae: 702.0654 - val_loss: 647.8571 - val_mae: 647.8571
Iteration: 1
39/39 [==============================] - 2s 60ms/step - loss: 576.4348 - mae: 576.4348 - val_loss: 492.1218 - val_mae: 492.1218
Iteration: 2
39/39 [==============================] - 2s 63ms/step - loss: 399.2912 - mae: 399.2912 - val_loss: 303.9146 - val_mae: 303.9146
Iteration: 3
39/39 [==============================] - 3s 67ms/step - loss: 270.9413 - mae: 270.9413 - val_loss: 255.0103 - val_mae: 255.0103
Iteration: 4
39/39 [==============================] - 3s 66ms/step - loss: 250.4909 - mae: 250.4909 - val_loss: 246.1546 - val_mae: 246.1546
Iteration: 5
39/39 [==============================] - 4s 90ms/step - loss: 237.3871 - mae: 237.3871 - val_loss: 228.7539 - val_mae: 228.7539
Iteration: 6
39/39 [==============================] - 3s 86ms/step - loss: 221.3777 - mae: 221.3777 - val_loss: 216.0527 - val_mae: 216.0527
Iteration: 7
